    16_230110_114_agg_combine_map
        

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

import warnings
warnings.filterwarnings(action='ignore') 

print(pd.__version__)
print(gpd.__version__)
print(folium.__version__)

1.4.4
0.12.2
0.14.0


In [2]:
#2. 프로젝트(1)  ... data load
#path = os.getenv('HOME') + '/aiffel/socar_gis/data/20210823_socar_zone.csv'
path = '/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/20210823_socar_zone.csv'

socar_zone_info_df = pd.read_csv(path)
socar_zone_info_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/20210823_socar_zone.csv'

In [ ]:
socar_zone_info_df[socar_zone_info_df['region1'] == '경기도']

In [ ]:
seoul_group_data = socar_zone_info_df.loc[socar_zone_info_df['region1'] == '경기도'].groupby('region2')['region2'].count()
seoul_group_data

    2. 프로젝트(1) 서울시 쏘카존 데이터를 활용한 시각화
       1) data load : socar zone data
       2) 시각화

In [ ]:
#2. 프로젝트(1)  # base nap : 서울시 중심부 위도, 경도
center = [37.541, 126.986]

map = folium.Map(
    location=center,
    zoom_start=10,
    )
print('슝~')

In [ ]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in socar_zone_info_df.iterrows():
    try:
        folium.Marker(
            location=[socar_zone_info_df.loc[idx, "lat"], socar_zone_info_df.loc[idx, "lng"]],
            popup=socar_zone_info_df.loc[idx, "zone_name"],
            icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

In [ ]:
#2. 프로젝트(1)  # map draw
map

    3. 프로젝트(2) 서울시 카페와 버스 정류장 밀도 데이터를 이용한 시각화
           1) 카페data 정리
              - data load
              - 카페data 전처리 : 업종data분류
              - 카페 위치데이터 리스트 만들기
           2) 버스data 정리
              - data load
              - 버스data 전처리 .. 컬럼명 변경
              - 버스 정류장 위치데이터 리스트 만들기
           3) 시각화 
              - Folium 초기 Map 객체 생성
              - 레이어 적용을 위한 그룹 만들기
              - 카페 밀도
              - 버스정류장 밀도
           4) 생성한 map 확인

In [ ]:
#3. 프로젝트(2) ... 1) 카페data load
temp_seoul_store_df = pd.read_csv(os. getenv('HOME') + '/aiffel/socar_gis/data/seoul_store_info_202106.csv')

temp_seoul_store_df.head(3)

In [ ]:
#3. 프로젝트(2) ... 1) 카페data 전처리 
seoul_cafe_store_df = temp_seoul_store_df[temp_seoul_store_df["상권업종소분류코드"] == "Q12A01"]

seoul_cafe_store_df.head()

In [ ]:
#3. 프로젝트(2) ... 1) 카페data 전처리 확인 
seoul_cafe_store_df.info()

In [ ]:
#3. 프로젝트(2) ... 1) 카페data 전처리 ... # 카페 위치데이터 리스트 만들기
seoul_cafe_location_data = [[row["위도"], row["경도"]] for _, row in seoul_cafe_store_df.iterrows()]

print("슝~")

In [ ]:
#3. 프로젝트(2) ... 2) 버스data load
path = os.getenv('HOME') + '/aiffel/socar_gis/data/seoul_bus_line_info.csv'
bus_line_info_df =  pd.read_csv(path)
bus_line_info_df.head()

In [ ]:
#3. 프로젝트(2) ... 2) 버스data 전처리 .. 컬럼명 변경
bus_line_info_df.rename(
    columns={
        "Y좌표": "lat",
        "X좌표": "lng",
    },
    inplace=True
)

bus_line_info_df.info()

In [ ]:
#3. 프로젝트(2) ... 2) 버스 정류장 위치데이터 리스트 만들기
bus_station_location_data = [[row["lat"], row["lng"]] for _, row in bus_line_info_df.iterrows()]
print('슝~')
#bus_station_location_data

In [ ]:
#3. 프로젝트(2) ... 3)시각화  ... # Folium 초기 Map 객체 생성 

center = [37.541, 126.986]

map = folium.Map(
    location=center,
    tiles= "Stamen Terrain", 
    zoom_start= 15
)
print('슝~')

In [ ]:
#3. 프로젝트(2) ... 3) 시각화 # 레이어 적용을 위한 그룹 만들기
# 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 카페 그룹 만들기
cafe = plugins.FeatureGroupSubGroup(fg, "cafe_density")
map.add_child(cafe)

# 버스정류장 그룹 만들기
bus = plugins.FeatureGroupSubGroup(fg, "bus_group")
map.add_child(bus)

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

In [ ]:
#3. 프로젝트(2) ... 3) 시각화 .. # 카페 밀도
plugins.HeatMap(seoul_cafe_location_data).add_to(cafe)

In [ ]:
#3. 프로젝트(2) ... 3) 시각화 ... # 버스정류장 밀도
plugins.HeatMap(bus_station_location_data).add_to(bus)

In [ ]:
#3. 프로젝트(2) ... 3) 시각화 ... # 생성한 맵 확인
map

    4. 프로젝트(3) 시각화 자료에 쏘카존 데이터 추가하기
        1) 초기 Map 객체를 생성
        2) 레이어 적용을 위한 그룹 만들기
        3) 쏘카존 마커 클러스터 만들기
        4) 카페 밀도
        5) 버스정류장 밀도
        6) 생성한 map 확인

In [ ]:
#4. 프로젝트(3) # 1)Folium 초기 Map 객체를 생성 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

map = folium.Map(
    location=center,
    zoom_start=10,
    )
print('슝~')

In [ ]:
#4. 프로젝트(3)  .. # 2)레이어 적용을 위한 그룹 만들기
# 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 카페 그룹 만들기
cafe = plugins.FeatureGroupSubGroup(fg, "cafe_density")
map.add_child(cafe)

# 버스정류장 그룹 만들기
bus = plugins.FeatureGroupSubGroup(fg, "bus_group")
map.add_child(bus)

# 쏘카 그룹 만들기
soacr = plugins.FeatureGroupSubGroup(fg, "socar_zone")
map.add_child(soacr)

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

In [ ]:
#4. 프로젝트(3)  ..3) 쏘카존 마커 클러스터 만들기
socar_zone_data = [[row["lat"], row["lng"]] for _, row in socar_zone_info_df.iterrows()]
popups = [socar_zone_info_df.loc[idx, "zone_name"] for idx, row in socar_zone_info_df.iterrows()] #popup시 표시 될 주소

mg = plugins.MarkerCluster( socar_zone_data, popups=popups,control=False)

map.add_child(mg)

In [ ]:
#4. 프로젝트(3)  ..#4) 카페 밀도
plugins.HeatMap(seoul_cafe_location_data).add_to(cafe)

In [ ]:
#4. 프로젝트(3)  ..#5) 버스정류장 밀도
plugins.HeatMap(bus_station_location_data).add_to(bus)

In [ ]:
#4. 프로젝트(3)  ..6) 생성한 map 확인

map

    5. socar_zone 분석 : 서울시 분석
       1) 서울시 쏘카존 밀도 분석   : 서초구 집중 >> 통,폐합 조정 필요
           - 산포도 확인 : bus, 카페, 쏘카존
           - 구별 쏘카존 밀도 분석 : 서초구 집중
       2) 서울시 구별 인구밀도 분석 : 
           - 강서구,성동구,송파구 인구밀도가 높음 
           - 쏘카존 신설 분석 필요
       3) 성동구기준 쏘카존 신설 분석 
          - 대단지 아파트에 설치된 쏘카존 분석 결과 다음의 아파트 쏘카존 신설 필요
            . 래미안 옥수 리버젠	 ( 서울특별시 성동구 매봉길 15 )
            . e편한세상 옥수 파크힐스( 서울특별시 성동구 매봉길 50 )

In [ ]:
#5 socar_zone 분석  .. bus scatter
bus_station_data = pd.DataFrame(bus_station_location_data,columns=['lat', 'lng'])

In [ ]:
#5 socar_zone 분석  .. bus scatter
ax1 = bus_station_data.plot.scatter(x='lng',y='lat',c='DarkBlue')

In [ ]:
#5 socar_zone 분석  .. caffe scatter
seoul_cafe__data = pd.DataFrame(seoul_cafe_location_data,columns=['lat', 'lng'])

In [ ]:
#5 socar_zone 분석  .. caffe scatter
ax1 = seoul_cafe__data.plot.scatter(x='lng',y='lat',c='DarkBlue')

In [ ]:
#5 socar_zone 분석  .. socar scatter
socar_data = pd.DataFrame(socar_zone_data,columns=['lat', 'lng'])

In [ ]:
#5 socar_zone 분석  .. socar scatter
ax1 = socar_data.plot.scatter(x='lng',y='lat',c='DarkBlue')

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 분석
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 지도 생성
map = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(map)

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 socar 밀도 확인
map = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(map)

map.choropleth(geo_data=seoul_geo,
             data=seoul_group_data, 
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 socar zone 수"
            )
map

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 인구 밀도 확인
path = os.getenv('HOME') + '/aiffel/000_going_node/004_221118_gonode4_jone/final_seoul_person.csv'

seoul_population_df = pd.read_csv(path)
seoul_population_df.head()

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 인구 밀도 sum
seoul_group_data = seoul_population_df.loc[seoul_population_df['gu'] != '서울특별시'].groupby('gu')['person'].sum()

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 인구 밀도 sum
seoul_group_data

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 인구 밀도 graph
map = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(map)

map.choropleth(geo_data=seoul_geo,
             data=seoul_group_data, 
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 인구 수"
            )
map
#강서구,성동구,송파구 sorcar zone 증설 필요성, 서초구 쏘카 통폐합 조정 필요

In [ ]:
#5 socar_zone 분석  .. 서울시 구별 인구 밀도 확인
path = os.getenv('HOME') + '/aiffel/000_going_node/004_221118_gonode4_jone/sungdong1.csv'

sungdong_df = pd.read_csv(path)
sungdong_df.head()

In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안 ### SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )
  
    access_token = temp_res.json()["result"]["accessToken"]
  
    return access_token

print("슝~")

In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안 ## SGIS access_token 가져오기
consumer_key  = '9e3111dfc4de422eac60'
consumer_secret = '6fa0e3a98f9143e89904'

access_token = get_access_token(consumer_key, consumer_secret)

## 주소를 입력해 주소의 데이터를 DataFrame으로 가지고오는 함수
def get_adress_info(access_token, address) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    temp_df = pd.DataFrame(temp_res)

    return temp_df

print("슝~")

In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안 ### UTM-K 좌표계를 WGS84 좌표계로 변환하는 함수
# 좌표계별 EPSG CODE 종류: UTM-K 좌표계(EPSG:5179), WGS84(위/경도 좌표계, EPSG:4326)

def utmk_to_wgs84(utmk_x, utmk_y) -> None:
    utmk_proj = pyproj.Proj(init="EPSG:5179") # UTM-K 좌표계 정의
    wgs84_proj = pyproj.Proj(init="EPSG:4326") # WGS84 좌표계 정의
    
    # 좌표 변환 수헹
    wgs84_x, wgs84_y = pyproj.transform(utmk_proj, wgs84_proj, utmk_x, utmk_y)
    # print(f"UTM-K 위도: {utmk_y} -> WGS84 위도: {wgs84_y} \nUTM-K 경도: {utmk_x} -> WGS84 경도: {wgs84_x}")
    
    
    return wgs84_x, wgs84_y

print("슝~")

In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안  ## SGIS 좌표변환 API를 활용한 좌표변환 함수 정의
def get_transcoord(access_token, src, dst, posX, posY) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌푯값 딕셔너리의 value 리턴

print("슝~")

In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안 .. 대단지 아파트 좌표 

sungdong_df.insert(2,"sido_nm", "nan")
sungdong_df.insert(3,"sgg_nm", "nan")
sungdong_df.insert(4,"adm_nm", "nan")
sungdong_df.insert(5,"x", "nan")
sungdong_df.insert(6,"y", "nan")
sungdong_df.insert(7,"wgs84_x", "nan")
sungdong_df.insert(8,"wgs84_y", "nan")
sungdong_df.insert(9,"transcoord_x", "nan")
sungdong_df.insert(10,"transcoord_y", "nan")

for i in range(len(sungdong_df)):
    address_df = get_adress_info(access_token, sungdong_df.loc[i]['address'] )

    sungdong_df.loc[i]["sido_nm"] = address_df.loc[0]["sido_nm"]
    sungdong_df.loc[i]["sgg_nm"]  = address_df.loc[0]["sgg_nm"]
    sungdong_df.loc[i]["adm_nm"]  = address_df.loc[0]["adm_nm"]
    sungdong_df.loc[i]["x"]       = address_df.loc[0]["x"]
    sungdong_df.loc[i]["y"]      = address_df.loc[0]["y"]
    
    # 변환후 데이터 프레임에 변환된 좌표 컬럼 추가하기
    sungdong_df.loc[i][["wgs84_x", "wgs84_y"]] = utmk_to_wgs84(sungdong_df.loc[i, "x"], sungdong_df.loc[i, "y"])
    sungdong_df.loc[i][["transcoord_x", "transcoord_y"]] = get_transcoord(access_token, "EPSG:5179", "EPSG:4326", sungdong_df.loc[0, "x"], sungdong_df.loc[0, "y"])

sungdong_df

In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안 ..  # Folium 초기 Map 객체 생성
# 초기 Map 메소드에는 초기 중심값 좌표가 들어가야 합니다.  
base_map = folium.Map(
    location=[37.566345, 126.977893]
)


In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안 .. 대단지 아파트 좌표  # 지도에 Circle 찍기
for index_num in range(len(sungdong_df)):
    folium.Circle(
        location=[sungdong_df.loc[index_num, "wgs84_y"], sungdong_df.loc[index_num, "wgs84_x"]],
        radius=100,
        color="blue",
        popup=sungdong_df.loc[index_num, "adm_nm"]
    ).add_to(base_map)

base_map

In [ ]:
#5 socar_zone 분석  .. 성동구 쏘카존 신설 제안 .. 대단지 아파트 와 쏘카존 비교 
marker_cluster = MarkerCluster().add_to(base_map)

#maker 객체 추가
for idx, row in socar_zone_info_df.iterrows():
    try:
        folium.Marker(
            location=[socar_zone_info_df.loc[idx, "lat"], socar_zone_info_df.loc[idx, "lng"]],
            popup=socar_zone_info_df.loc[idx, "zone_name"],
            icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")            
        ).add_to(marker_cluster)
    except:
        pass



In [ ]:
# base_map